In [15]:
from gensim.utils import tokenize
import pandas as pd
import numpy as np
import sys
import os
import liwc_utils
from collections import defaultdict
df = pd.read_csv("/Users/Lara/Dropbox/LanguageGender/Data/Advocates/Cleaned/cleaned_Advocates_Community.csv")
liwc_cats = ['Incl', 'Article', 'Inhib', 'You', 'Present', 'Social', 'We', 'Feel', 'Ingest', 'Time', 'Cause', 'Leisure', 'Quant', 'Money', 'Anx', 'Assent', 'Conj', 'Family', 'Negate', 'CogMech', 'Achiev', 'See',
    'Relig', 'Certain', 'Ipron', 'Tentat', 'Hear', 'Percept', 'AuxVb', 'Nonflu', 'Pronoun', 'Relativ', 'Bio', 'Friends', 'Work', 'I', 'Negemo', 'Anger', 'Body', 'SheHe', 'Ppron', 'Affect', 'Health', 'Discrep', 'Motion',
    'Sad', 'Posemo', 'Humans', 'Home', 'Swear', 'They', 'Future', 'Excl', 'Filler', 'Space', 'Death', 'Prep', 'Past', 'Sexual', 'Insight', 'Funct', 'Adverbs', 'Numbers', 'Verbs']


In [16]:
def compute_avg(col):
    female = defaultdict(int)
    male = defaultdict(int)
    num_female = 0
    num_male = 0
    liwc_df = []
    for index, row in df.iterrows():
        l_dict = liwc_utils.response_to_liwc(row[col])
        liwc_row = [row['ResponseId']]
        for c in liwc_cats:
            liwc_row.append(l_dict.get(c, 0))
        liwc_df.append(liwc_row)
        
        if row['Gender'] == 'Male':
            for cat, val in l_dict.items():
                male[cat] += val
            num_male += 1
        elif row['Gender'] == 'Female':
            for cat, val in l_dict.items():
                female[cat] += val
            num_female += 1

    for cat, val in female.copy().items():
        female[cat] = val / num_female
    for cat, val in male.copy().items():
        male[cat] = val / num_male

    liwc_df = pd.DataFrame(liwc_df, columns=['ResponseId']+liwc_cats)
    return female, male, liwc_df
        

In [17]:
female_q1, male_q1, liwc_df = compute_avg("Intro.Question.1")
new_df = pd.merge(df, liwc_df)

female_q2, male_q2, liwc_df = compute_avg("Intro.Question.2")
new_df = pd.merge(new_df, liwc_df, on=['ResponseId'], suffixes=['_q1', '_q2'])

female_exp, male_exp, liwc_df = compute_avg("text")
new_df = pd.merge(new_df, liwc_df, on=['ResponseId'])


In [18]:
new_df.rename(columns={c:c+"_exp" for c in liwc_cats}, inplace=True)

In [19]:
rows = []
for c in liwc_cats:
    rows.append([female_q1[c], male_q1[c], female_q2[c], male_q2[c], female_exp[c], male_exp[c]])

In [20]:
liwc_avgs = pd.DataFrame(rows, index=liwc_cats, columns=['Female_Q1', 'Male_Q1', 'Female_Q2', 'Male_Q2', 'Female_Exp', 'Male_Exp'])

In [21]:
liwc_avgs['q1_diff'] = liwc_avgs['Female_Q1'] - liwc_avgs['Male_Q1']
liwc_avgs['q2_diff'] = liwc_avgs['Female_Q2'] - liwc_avgs['Male_Q2']
liwc_avgs['exp_diff'] = liwc_avgs['Female_Exp'] - liwc_avgs['Male_Exp']


In [22]:
liwc_avgs

,Female_Q1,Male_Q1,Female_Q2,Male_Q2,Female_Exp,Male_Exp,q1_diff,q2_diff,exp_diff
Incl,1.747475,1.673267,0.838384,0.811881,2.404040,2.524752,0.074207,0.026503,-0.120712
Article,1.474747,1.762376,1.303030,1.594059,1.414141,1.861386,-0.287629,-0.291029,-0.447245
Inhib,0.090909,0.059406,0.090909,0.059406,0.131313,0.079208,0.031503,0.031503,0.052105
You,0.333333,0.405941,0.131313,0.168317,0.242424,0.237624,-0.072607,-0.037004,0.004800
Present,1.676768,1.584158,1.161616,0.762376,2.282828,2.356436,0.092609,0.399240,-0.073607
...,...,...,...,...,...,...,...,...,...
Insight,1.010101,0.940594,0.242424,0.267327,0.727273,0.574257,0.069507,-0.024902,0.153015
Funct,10.171717,10.267327,8.020202,7.792079,13.464646,15.029703,-0.095610,0.228123,-1.565057
Adverbs,0.454545,0.712871,0.414141,0.405941,0.737374,0.663366,-0.258326,0.008201,0.074007
Numbers,0.161616,0.128713,0.060606,0.029703,0.151515,0.168317,0.032903,0.030903,-0.016802


In [23]:
liwc_avgs[['Female_Q1', 'Male_Q1', 'q1_diff']].sort_values(by='q1_diff')


,Female_Q1,Male_Q1,q1_diff
Article,1.474747,1.762376,-0.287629
Adverbs,0.454545,0.712871,-0.258326
Relativ,2.010101,2.217822,-0.207721
Space,1.494949,1.673267,-0.178318
Funct,10.171717,10.267327,-0.095610
...,...,...,...
Leisure,0.191919,0.099010,0.092909
Achiev,0.787879,0.683168,0.104710
CogMech,3.838384,3.722772,0.115612
Prep,2.777778,2.623762,0.154015


In [24]:
liwc_avgs[['Female_Q2', 'Male_Q2', 'q2_diff']].sort_values(by='q2_diff')


,Female_Q2,Male_Q2,q2_diff
Article,1.303030,1.594059,-0.291029
Conj,0.848485,1.108911,-0.260426
Achiev,0.464646,0.702970,-0.238324
Cause,0.151515,0.237624,-0.086109
Leisure,0.262626,0.336634,-0.074007
...,...,...,...
Verbs,1.535354,1.306931,0.228423
Prep,2.525253,2.277228,0.248025
Quant,0.656566,0.396040,0.260526
Relativ,2.575758,2.227723,0.348035


In [26]:
liwc_avgs[['Female_Exp', 'Male_Exp', 'exp_diff']].sort_values(by='exp_diff')


,Female_Exp,Male_Exp,exp_diff
Funct,13.464646,15.029703,-1.565057
Pronoun,3.666667,4.118812,-0.452145
Article,1.414141,1.861386,-0.447245
Ipron,1.454545,1.851485,-0.396940
Verbs,3.121212,3.445545,-0.324332
...,...,...,...
Adverbs,0.737374,0.663366,0.074007
Feel,0.212121,0.099010,0.113111
Achiev,1.151515,1.009901,0.141614
Insight,0.727273,0.574257,0.153015


In [27]:
new_df.to_csv("/Users/Lara/Dropbox/LanguageGender/Data/Advocates/Cleaned/liwc_community.csv", index=False)

In [13]:
new_df

,ResponseId,Intro.Question.1,Intro.Question.2,Education,Age,Gender,Gender_3_TEXT,Attention.Check,Ethnicity,Ethnicity_6_TEXT,...,Space_exp,Death_exp,Prep_exp,Past_exp,Sexual_exp,Insight_exp,Funct_exp,Adverbs_exp,Numbers_exp,Verbs_exp
0,R_3oLW1PiBnOLJy0U,Fast and efficient,Quickly clearing a repaired queue,Some college,32,Male,NaN,Purple,White or Caucasian,NaN,...,1,0,1,1,0,1,18,3,0,4
1,R_3PSR8Y1s3pcp5DG,"Quick, fast moving",racing,Graduate degree,35,Male,NaN,Purple,White or Caucasian,NaN,...,1,0,3,0,0,0,9,0,0,0
2,R_r7GlNYxUGxOiVnr,The rate at which work is done,Resolving customer escalations as soon as they...,Bachelor degree,26,Male,NaN,Purple,Asian or Asian American,NaN,...,0,0,1,0,0,1,7,0,0,2
3,R_ylN0QOOQMfzqdjP,Speed is the quickness in pace that a task is ...,Running,Graduate degree,30,Female,NaN,Purple,White or Caucasian,NaN,...,1,0,1,0,0,0,4,0,0,1
4,R_2bZRUH2Kz2OmAJb,Speed isn't a value it's just a thing that the...,Running from meeting to meeting.,Graduate degree,30,Female,NaN,Purple,White or Caucasian,NaN,...,2,0,5,0,0,0,14,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,R_s6VmmcCkWPI8rD3,Being fast and prompt.,Giving information and service rapidly.,Some college,35,Male,NaN,Purple,White or Caucasian,NaN,...,0,0,0,1,0,1,3,0,0,2
196,R_bqRcPNVF2zKXzdD,I would think it means how quickly tasks are c...,An employee is tasked with coming up with suit...,Some college,61,Female,NaN,Purple,Black of African American,NaN,...,1,0,7,1,0,1,15,0,0,2
197,R_2zTr2YohZPoKNcj,Speed means how effective and fast something is.,Finishing a task in an hour that would usually...,Some college,32,Female,NaN,Purple,Asian or Asian American,NaN,...,1,0,1,0,0,0,4,0,0,1
198,R_1JKNvlMbtovhzdY,quick turnaround of ideas to product rollout,implementing feedback from customers quickly\n,Bachelor degree,22,Female,NaN,Purple,Latino or Latina,NaN,...,1,0,2,0,0,1,8,0,0,3
